In [ ]:
from termcolor import colored
import jsonlines

In [ ]:
max_duplicates = "--max_duplicates 10" # if true, each entity appears only once in the entire dataset.
prefix = "--prefix march_" # suffix for the output files
version_name = "--version_name with_neg"
wandb_project = "--wandb_project auto-limit"
experiment_suffix = "--experiment_suffix march"
superclass_tag = "--superclass_tag ORG"
target_tag = "--target_tag SCHOOL"
patterns_file = "--patterns school_patterns.json"
dataset_name = "--dataset_name schools"
dataset_path= f'--dataset_path data/schools'

In [ ]:
!python ./src/tag_dataset.py -fn results -d schools -t SCHOOL --include_only_o --prefix "no_org"

In [ ]:
!python src/train.py --dataset schools {prefix} {superclass_tag} --experiment march --show_on_wandb --epochs 3

In [ ]:
!python src/collect_data.py {max_duplicates} {prefix} {superclass_tag} {patterns_file}

In [ ]:
# make sure the spike matches make sense
! head -5 data/spike_matches/positive/0_schools.jsonl

In [ ]:
!python src/tag_dataset.py {prefix} {version_name} {superclass_tag} {target_tag} {dataset_name}

In [ ]:
# make sure the tagged data makes sense
with jsonlines.open("./data/schools/dataset_expand.jsonl", 'r') as f:
    for i, line in enumerate(f):
        color_text = [x[0] if x[1] == "O" else colored(x[0], 'red') if "ORG" in x[1] else colored(x[0], 'green') for x in line["sent_items"]]
        print(i, line["id"], " ".join(color_text))
        print()

In [ ]:
!python src/train.py {wandb_project} {experiment_suffix} {version_name}  {superclass_tag} {target_tag} {dataset_path} --epochs 3

In [ ]:
!python src/evaluation.py {experiment} {version_name}  {superclass_tag} {target_tag} {dataset_path}

In [ ]:
!python src/evaluation.py -d schools  -t SCHOOL --experiment march --prefix march_ --eval_on_test

In [ ]:
!python src/evaluation.py -d schools  -t SCHOOL --experiment march --prefix march_ --eval_on_entire_set --eval_on_test

In [ ]:
f"python src/evaluation.py {wandb_project} {experiment_suffix} {version_name}  {superclass_tag} {target_tag} {dataset_path}"

In [ ]:
%matplotlib inline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

LABELS = [f"B-SCHOOL",
          f"I-SCHOOL",
          "O"]
matrix = """[[0.818 0.013 0.169]
 [0.005 0.876 0.118]
 [0.001 0.002 0.997]]"""
matrix = matrix.replace("\n", ",").replace("   ", ",").replace("  ", ",").replace(", ", ",").replace(" ", ",")
matrix = np.array(matrix)
print(matrix, type(matrix))
matrix = matrix.tolist()
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=LABELS)
disp.plot()
plt.show()
# disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=LABELS)
# disp.plot()
# plt.show()

In [ ]:
matrix = np.array([[0.818,0.013,0.169],[0.005,0.876,0.118],[0.001,0.002,0.997]])
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=LABELS)
disp.plot()
plt.show()

In [ ]:
[[0.883,0.083,0.,0.,0.033],[0.051,0.763,0.,0.021,0.165],[0.019,0.009,0.833,0.06,0.079],[0.,0.,0.078,0.773,0.149],[0.001,0.012,0.005,0.011,0.972]]